In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
#Turn off warnings because hyper-parameter tuning will test many poor performing models
from sys import warnoptions
from warnings import simplefilter
import os
if not warnoptions:
    simplefilter("ignore")
os.environ["PYTHONWARNINGS"] = "ignore" 

In [3]:
from vb_helper import VBHelper

kwargs=dict(
    shuffle=True,
    drop_duplicates='Xy',#'X',False
    nan_threshold=0.9,#drop any column with more than this share of nulls
    run_stacked=True, 
    test_share=0,
    cv_folds=5,
    cv_reps=10,
    cv_strategy=None,# ('quantile',5), ('uniform',5) # (stratification method, groupcount)
    )
vbhelper=VBHelper(**kwargs)

In [4]:
#load data
data_path=os.path.join('sample_data','ex1.csv')
df=pd.read_csv(data_path)
df.index=[str(i) for i in df.index]
#select variables
y_name='THG_Fish'
loc_vars=['LAT','LONG']
drop_vars=['Date','OrigHabCode','STA_ID']
drop_vars.extend(loc_vars)
drop_vars.append(y_name)
all_vars=list(df.columns)
x_vars=[var for var in all_vars if var not in drop_vars]

#extract random rows for prediction
predict_n=30
shuf=np.arange(df.shape[0])
seed=0
rng = np.random.default_rng(seed)
rng.shuffle(shuf)
predict_select=shuf[:predict_n]
X_predict=df.loc[:,x_vars].iloc[predict_select]
y_predict=df.loc[:,y_name].iloc[predict_select]
predict_loc=[df.index[i] for i in predict_select]
df.drop(index=predict_loc,inplace=True)
X_df=df.loc[:,x_vars]
y_df=df.loc[:,[y_name]]
print(f'indices for predictions: {X_predict.index.to_list()}')

indices for predictions: ['420', '652', '575', '89', '23', '19', '2', '273', '495', '585', '737', '707', '600', '162', '539', '94', '100', '725', '335', '251', '672', '261', '643', '766', '36', '49', '763', '398', '742', '84']


In [5]:
scorer_list=['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2'] #strings defined in scikit-learn
vbhelper.scorer_list=scorer_list

In [6]:
y_df.shape

(744, 1)

In [7]:
vbhelper.setData(X_df,y_df)

# of duplicate rows of data: 0
# of duplicate rows of X: 0
# of duplicate Xy rows dropped: 0
no columns exceeded nan threshold of 0.9


In [8]:
inner_cv_dict={
    'cv_reps':1,
    'cv_folds':5,
    'cv_strategy':('quantile',5)} # ensure each fold has y values from each quantile
inner_cv=vbhelper.getCV(cv_dict=inner_cv_dict)

In [9]:
prep_dict={
    'cat_approach':'together', # imputation is over all variables after one-hot-encoding
    'impute_strategy':'IterativeImputer', # python implementation of MICE: Multivariate Imputation by Chained Equations in R”
    'cat_idx':vbhelper.cat_idx # keep track of the categorical variables
    }
pipe_dict={} # the pipeline setup will go here

In [10]:
pipe_kwargs={
    'do_prep':not vbhelper.run_stacked, # the stacking regressor will do imputation if run_stacked==True
    'prep_dict':prep_dict,
    'inner_cv':inner_cv,
    'cat_idx':vbhelper.cat_idx,
    'float_idx':vbhelper.float_idx,
    'bestT':False # if true, test each covariate for optimal transformation 
    }

In [11]:
from vb_estimators import GBR

gbr_kwargs=pipe_kwargs.copy()
gbr_kwargs['est_kwargs']={
    'n_estimators':[64,128],
    'max_depth':[2,3]
    }
pipe_dict['gbr']={
    'pipe':GBR,
    'pipe_kwargs':gbr_kwargs
    }

In [12]:
from vb_estimators import  L1Lars

l1_kwargs=pipe_kwargs.copy()
l1_kwargs['max_n_alphas']=500 # alpha is the only hyper-parameter
pipe_dict['lassolars']={
    'pipe':L1Lars,
    'pipe_kwargs':l1_kwargs
    }

In [13]:
from vb_estimators import RBFSVR

rbf_kwargs=pipe_kwargs.copy()
rbf_kwargs['gridpoints']=5 
pipe_dict['rbfSVR']={
    'pipe':RBFSVR,
    'pipe_kwargs':rbf_kwargs
    }

In [14]:
from vb_estimators import FlexiblePipe

nl_pipe_kwargs=pipe_kwargs.copy()
nl_pipe_kwargs['functional_form_search']=True
nl_pipe_kwargs['flex_kwargs']={'robust':True}
pipe_dict['nonlinear']={
    'pipe':FlexiblePipe,
    'pipe_kwargs':nl_pipe_kwargs
}

In [15]:
#and load the pipelines
vbhelper.setPipeDict(pipe_dict)
vbhelper.setModelDict()

In [16]:
from sklearn.model_selection import train_test_split
from time import time
X_train, X_test, y_train, y_test=train_test_split(X_df,y_df,test_size=0.7,random_state=5) #just for debugging

In [17]:
runtest=False
if runtest:
    for name,est in vbhelper.model_dict.items():
        start=time()
        print(name)
        est.fit(X_train,y_train)
        print('train R2:',est.score(X_train,y_train))
        if not X_test is None:print('test R2:',est.score(X_test,y_test))
        end=time()
        print(f'runtime:{(end-start)/60} min.\n')

In [18]:
if vbhelper.run_stacked and runtest:
    mp=vbhelper.model_dict['multi_pipe']
    fitted_ipipe_dict=mp.build_individual_fitted_pipelines()
    for p_name,p in fitted_ipipe_dict.items():
        print(f'{p_name} scored: {p.score(X_test,y_test)}')


In [ ]:
vbhelper.runCrossValidate(try_load=True)

jhash:  908f54e804fa86e92d794e778b658501


In [ ]:
vbhelper.buildCVScoreDict()
vbhelper.saveCVResults()

### Plot the results

In [ ]:
from vb_plotter import VBPlotter
plotter=VBPlotter()

with open('project_cv_results.json','rb') as f:
    cv_results_and_scores=json.load(f)
plotter.setData(cv_results_and_scores)

In [ ]:
#plotter.plotCVYhatVsY(single_plot=True,include_all_cv=True)

In [ ]:
#plotter.plotCVYhat(single_plot=True,include_all_cv=True) #add option to use original row order instead of sort

In [ ]:
#plotter.plotCVScores(sort=1)

In [ ]:
#plotter.plotBoxWhiskerCVScores()

# VB web
## Small Data Procedure

#### Task 2: Train the Final, Predictive Model
##### Using all of the data

In [ ]:
vbhelper.refitPredictiveModel(selected_model='stacking_reg')

# VB web
## Small Data Procedure
### Predict Phase
Finally we are ready to use the predictive model that we just estimated.


In [ ]:
vbhelper.predictandSave(X_predict)

In [ ]:
print(vbhelper.PIdf)
pd.read_json(vbhelper.PIdf.to_json())

In [ ]:
with open('project_prediction_results.json', 'r') as f:
    project_prediction_results=json.load(f)
plotter.setPredictData(project_prediction_results)

In [ ]:
p_kwargs=dict(single_plot=True, include_all_cv=True,
              ypredict=True,cv_ypredict=True,estimators='selected')
plotter.plotCVYhatVsY(**p_kwargs)

In [ ]:
plotter.yhat_stack_dict['gbr'].shape

#### And redraw the plots with the known values of y as vertical lines

In [ ]:
plotter.plotCVYhatVsY(**p_kwargs,true_y=y_predict)

#### Or we can predict each observation separately

In [ ]:
plotters=[VBPlotter() for _ in range(predict_n)]
for p in plotters: p.setData(cv_results_and_scores)
for i in range(predict_n): plotters[i].setPredictData(project_prediction_results,loc_row=y_predict.index[i])
for i in range(predict_n): plotters[i].plotCVYhatVsY(**p_kwargs,true_y=y_predict.loc[[y_predict.index[i]]])